# URDF experiments for design on demand

URDF units are written and joined with the help of the ODIO URDF library and constructed robots are played around with using the teleop from pydrake

In [1]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output
import plotly.express as px

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, InverseKinematics,
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MeshcatVisualizerCpp, MultibodyPlant, MultibodyPositionToGeometryPose, Parser,
    PiecewisePose, PiecewisePolynomial, Quaternion, RigidTransform, Solve,
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSlidersThatPublish, MeshcatPoseSliders)
from manipulation.scenarios import AddMultibodyTriad, SetColor

import pandas as pd

In [2]:
from odio_urdf import *
import math
import sys
# from operator import add
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

In [3]:
meshcat = StartMeshcat()

In [66]:
# Initiate the tree structure with a base

# Decided to use numpy for array manipulation

# Need to setup good connection rules for both origin and orientation
# And then setup joints

# We need a new class to describe the links and the joints
# The end product has to be disctributed as modules and submodules
# https://stackoverflow.com/questions/18555193/python-modules-with-submodules-and-functions

# For now we have the module class within the main class
class robotDoD:
    # A class to define all the modules and their description
    class robotModule:
        def __init__(self, name):
            self.name = name;
            # Define interfaces poition and orientation
            self.intA = np.array([[0,0,0],[0,0,0]]) # Interface A origin and orientation
            self.intB = np.array([[0,0,0],[0,0,0]]) # Interface B origin and orientation
            self.mesh_path = "" # Set the location of the path
            self.scale = "0.001 0.001 0.001" # Set scale as string
            self.type = 0 # 0-link; 1-joint    
#             self.default_orientation = np.array([0, 0, 0]) # If necessary to re-orient from the obtained meshes
#             self.default_origin = np.array([0,0,0]) # To re-locate from the default location of the mesh
  
    # A helper class for transformation
    class helperFuncs:
        def __init__(self):
            pass
            
        def list2space(self, com):
            return str(com[0])+' '+str(com[1])+' '+str(com[2])
    
        def rotZ(self, angle):
            return np.array([[np.cos(angle), -np.sin(angle), 0],[np.sin(angle), np.cos(angle), 0],[0,0,1]])

        def rotY(self, angle):
            return np.array([[np.cos(angle), 0, np.sin(angle)],[0, 1, 0],[-np.sin(angle), 0, np.cos(angle)]])

        def rotX(self, angle):
            return np.array([[1,0,0], [0, np.cos(angle), -np.sin(angle)],[0, np.sin(angle), np.cos(angle)]])
    
        def transform_vector(self, axis, angle, vector):
            if axis == 0:
                return self.rotX(angle).dot(vector)
            if axis == 1:
                return self.rotY(angle).dot(vector)
            if axis == 2:
                return self.rotZ(angle).dot(vector)
            else:
                raise AssertionError("Error: Rotation about unknown axis requested") 
                
        # Resolve the axis and angle when aligned
        def resolve_axis(self, module):
            def foo(x):
                return x[0]*x[1]*x[2]

            def non_lin(x):
                return np.linalg.norm(x)

            lin_constraint = LinearConstraint(module.intA[1], 0, 0)
            nonlin_constraint = NonlinearConstraint(non_lin, 1, 1)

            sol = minimize(foo,[1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3)], constraints = [lin_constraint, nonlin_constraint])
            return sol['x']            
    
    def __init__(self, name):
        self.name = name
        # Initiate the helper class
        self.helper_funcs = self.helperFuncs()
        # Declare materials
        self.materials = Group(
            Material("blue", Color(rgba="0 0 0.8 1.0")),
            Material("red", Color(rgba="0.8 0 0 1.0")),
            Material("black", Color(rgba="0.0 0.0 0.0 1.0")))  
        self.material_color = ["blue", "red", "black"]
        self.module_types = ["linkMod", "jointMod"]
        # Declare the robot
        self.robot = Robot(self.name, self.materials)
        # Declare robots tree structure
        self.tree = {}
        self.prev_module = []
        
    def add_root(self):        
        # Add root module to the robot URDF
        root_mod = self.robotModule('root_mod')
        # Define the module
        root_mod.intA = np.array([[0,0,0],[0,0,-1]])
        root_mod.intB = np.array([[0,0,0.01],[0,0,1]])
        root_mod.type = -1
        
        root_link_urdf = Link(
                    Visual(
                        Origin(rpy = "0 0 0", xyz = self.helper_funcs.list2space(root_mod.intA[0])),
                        Material (name = "black"),
                        Geometry(Cylinder(length = 0.02, radius = 0.05))),
                    name = root_mod.name)
        
        self.prev_module = root_mod.intB
        self.robot(root_link_urdf)
        self.tree[root_mod.name] = root_mod.intB
        
        # Weld root to the world
#         Weld robot seperately later in pydrake
#         joint_type = "fixed"
#         joint_name  = 'world_weld'
#         root_weld = Joint(
#                         Parent("world"),
#                         Child("root_mod"),
#                         Origin(xyz = self.helper_funcs.list2space(root_mod.intA[0]), rpy = self.helper_funcs.list2space(root_mod.intA[1])),
#                         Axis(xyz = self.helper_funcs.list2space(root_mod.intB[1])),
#                         type = joint_type,
#                         name= joint_name)

#         self.robot(root_weld)
    
    def add_module(self, module):
        # Always start from the root module which initates the self.prev_module
        # Check if prev_module is empty => root not initiated 
        if not self.prev_module.any():
            raise AssertionError("Error: Add root module first before initiating the tree")
        # Get info from self.prev_module
        # Compute connection rules from prev and current modules
        current_origin, current_orientation = self.computeConnection(module)
        
        link_name = self.module_types[module.type]+ '_'+str(len(self.tree.keys()))
        # Generate module URDF
        module_link_urdf = Link(
                         Visual(
                            Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space(current_orientation)),
#                              Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
#                             Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space([0,0,0])),
#                             Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space(current_orientation)),
                            Material(name = self.material_color[module.type]),
                            Geometry(Mesh(filename = module.mesh_path, scale = module.scale))),
                        name= link_name)
        
        # Add the module to the robot URDF
        self.robot(module_link_urdf)

        # Now add joints everytime a new module is added
        # If link is added then add a revolute joint to the previous module
        # If a joint is added then add a fixed joint with the previous module
        
        # TODO!!!
        # ORIGIN FOR ALL THE JOINT MODULES IS AT O. CHANGE THIS!!
        
        
        if module.type == -1:
            # Do nothing if root            
            pass
        
        if module.type == 0: # If link
            joint_type = "revolute"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space([0,0,0])),
#                 Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
#                                     Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space([0,0,0])),
#                                     Origin(xyz = self.helper_funcs.list2space(next_origin), rpy = self.helper_funcs.list2space(next_orientation)),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)

            self.robot(module_joint_urdf)
            
            # If revolute then also add an actuator here
            actuator_name = 'act_'+joint_name
            module_actuator_urdf = Transmission(Type("SimpleTransmission"),
                                    Actuator(Mechanicalreduction('1'), name = actuator_name),
                                    Transjoint(joint_name),
                                    name = actuator_name)
            # Check with this actuator and see if it works
#             print(module_actuator_urdf)
            self.robot(module_actuator_urdf)
            
            
        if module.type == 1 and list(self.tree.keys())[-1] != "root_mod": # If joint
            joint_type = "fixed"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space([0,0,0])),
                    #                 Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
#                                     Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space([0,0,0])),
#                                     Origin(xyz = self.helper_funcs.list2space(next_origin), rpy = self.helper_funcs.list2space(next_orientation)),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)
            
            self.robot(module_joint_urdf)
        
        if module.type == 1 and list(self.tree.keys())[-1] == "root_mod": # If joint and previous is root
            print(self.prev_module)
            joint_type = "revolute"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space([0,0,0])),
#                 Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
#                                     Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space([0,0,0])),
#                                     Origin(xyz = self.helper_funcs.list2space(next_origin), rpy = self.helper_funcs.list2space(next_orientation)),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)

            self.robot(module_joint_urdf)
            
            # If continuous then also add an actuator here
            actuator_name = 'act_'+joint_name
            module_actuator_urdf = Transmission(Type("SimpleTransmission"),
                                    Actuator(Mechanicalreduction('1'), name = actuator_name),
                                    Transjoint(joint_name),
                                    name = actuator_name)
            # Check with this actuator and see if it works
            # print(module_actuator_urdf)
            self.robot(module_actuator_urdf)
        
        # Save all the origins and orientations in a tree
        self.prev_module = self.new_prev_module
        self.tree[link_name] = self.prev_module

    def constructURDF(self):
        # Later replace the name of the URDF to the given name
        name = 'test.urdf'
        with open(name, 'w') as f:
            print(self.robot, file=f)
            print('Robot design successfully saved as '+name)
    
    def visualiseRobot(self, urdf_path, meshcat):
#         meshcat = StartMeshcat()
        # Loading the updated robot URDF
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=0.001)
        self.model = Parser(self.plant).AddModelFromFile(urdf_path)
        self.plant.Finalize()
        
        meshcat.Delete()
        meshcat.DeleteAddedControls()
        
        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(self.builder, self.scene_graph, meshcat)
        
        self.diagram = self.builder.Build()
        self.diagram_context = self.diagram.CreateDefaultContext()
        self.diagram.Publish(self.diagram_context)
        
    def computeConnection(self, module):
        current_orientation = np.zeros(3)
        # Compute the rotation axis
        temp = np.cross(self.prev_module[1], module.intA[1])
        sinangle = np.linalg.norm(temp)
        cosangle = np.round(np.dot(self.prev_module[1], module.intA[1]))
        # Hangle sinangle being zero
        # print(module.name)
        if sinangle == 0 and cosangle == 1:
            # We need the vector flipped
            # We need to resolve redundancy here by solving an optimisation problem
            rotaxis = self.helper_funcs.resolve_axis(module)
            rotangle = np.pi
            
        if sinangle == 0 and cosangle == -1:
            rotangle = 0
            rotaxis = np.zeros(3)
            
        if sinangle != 0:
            rotaxis = temp/sinangle
            # Compute the rotation angle
            rotangle = np.arctan2(sinangle, cosangle)
        
        # print(module.name,' ', rotaxis, ' ', rotangle)
        # Compute the quaternion
        r = R.from_rotvec(rotangle*rotaxis)
        # Convert to Euler rotation 'xyz'
        current_orientation = r.as_euler('xyz')

        # Rotation matrix
        rmat = r.as_matrix()
        current_origin = self.prev_module[0] + np.dot(rmat,(-module.intA[0]))
        print(current_origin)
#         current_origin = self.prev_module[0] + np.dot(rmat,(module.intA[0]))
        # Next orientation
        next_orientation = np.dot(rmat, module.intB[1])
        next_origin = current_origin + np.dot(rmat, module.intB[0])
        # Update prev_module        
        self.new_prev_module = np.array([next_origin, np.round(next_orientation)])
    
        return current_origin, current_orientation

In [67]:
# Declare the robotDoD class
roboDoD = robotDoD("test");

# Declare all the modules
# Keep this database of modules independant of the experiment being done

# jmod
jmod = robotDoD.robotModule('jmod')
# Define the module
# jmod.intA = np.array([[0,0,0],[0,-1,0]])
# jmod.intB = np.array([[0.031, 0.0556, 0],[1,0,0]])
jmod.intA = np.array([[0,-0.0035,0],[0,-1,0]])
jmod.intB = np.array([[0.03, 0.0495, 0.0003],[1,0,0]])
jmod.type = 1
jmod.mesh_path = "./meshes/j_module.obj"

# l1mod
l1mod = robotDoD.robotModule('l1mod')
# Define the module
l1mod.intA = np.array([[0.04,0,0],[1,0,0]])
l1mod.intB = np.array([[0.035,0.16,0],[1,0,0]])
l1mod.type = 0
l1mod.mesh_path = "./meshes/l1_module.obj"

# l2mod
l2mod = robotDoD.robotModule('l2mod')
# Define the module
l2mod.intA = np.array([[0.04,0,0],[1,0,0]])
l2mod.intB = np.array([[0,0.148,0],[0,1,0]])
l2mod.type = 0
l2mod.mesh_path = "./meshes/l2_module.obj"

# l3mod
l3mod = robotDoD.robotModule('l3mod')
# Define the module
l3mod.intA = np.array([[0,0.148,0],[0,1,0]])
l3mod.intB = np.array([[0.04, 0, 0],[1,0,0]])
l3mod.type = 0
l3mod.mesh_path = "./meshes/l3_module.obj"

In [68]:
# Now we have a base robotDoD class and now adding modules one by one
roboDoD.add_root()
roboDoD.add_module(jmod)
roboDoD.add_module(l2mod)
# roboDoD.add_module(jmod)
# roboDoD.add_module(l1mod)
# roboDoD.add_module(jmod)
# roboDoD.add_module(l2mod)
# roboDoD.add_module(jmod)
# roboDoD.add_module(l3mod)
# roboDoD.add_module(jmod)
# roboDoD.add_module(l1mod)
# roboDoD.add_module(jmod)
# roboDoD.add_module(l3mod)

# Now construct the URDF
roboDoD.constructURDF()

# Loading the updated robot URDF
meshcat.Delete()
roboDoD.visualiseRobot("./test.urdf", meshcat)

[2021-12-14 13:58:54.201] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/meshes/l2_module.obj references f7b2bd91-5f44-436c-8d02-ef081fb01593.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/meshes/f7b2bd91-5f44-436c-8d02-ef081fb01593.mtl"
[2021-12-14 13:58:54.226] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/meshes/j_module.obj references 29fd947b-97f8-4696-91a7-40d64dd5beae.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/meshes/29fd947b-97f8-4696-91a7-40d64dd5beae.mtl"


[[0.   0.   0.01]
 [0.   0.   1.  ]]
Robot design successfully saved as test.urdf


In [69]:
len(roboDoD.tree);

In [70]:
roboDoD.robot

<robot  name="test" >
 <material  name="blue" >
  <color  rgba="0 0 0.8 1.0" />
 </material>
 <material  name="red" >
  <color  rgba="0.8 0 0 1.0" />
 </material>
 <material  name="black" >
  <color  rgba="0.0 0.0 0.0 1.0" />
 </material>
 <link  name="root_mod" >
  <visual >
   <origin  rpy="0 0 0"  xyz="0 0 0" />
   <material  name="black" />
   <geometry >
    <cylinder  radius="0.05"  length="0.02" />
   </geometry>
  </visual>
 </link>
 <link  name="jointMod_1" >
  <visual >
   <origin  rpy="1.5707963267948963 0.0 0.0"  xyz="0 0 0" />
   <material  name="red" />
   <geometry >
    <mesh  filename="./meshes/j_module.obj"  scale="0.001 0.001 0.001" />
   </geometry>
  </visual>
 </link>
 <joint  type="revolute"  name="joint_1_revolute" >
  <parent  link="root_mod" />
  <child  link="jointMod_1" />
  <origin  rpy="0 0 0"  xyz="0.0 -7.771561172376096e-19 0.006500000000000001" />
  <axis  xyz="0.0 0.0 1.0" />
 </joint>
 <transmission  name="act_joint_1_revolute" >
  <type  xmltext="Sim

In [ ]:
# Verify generated URDF using ROS URDF verifyer
# Manipulate robot FK
# Manipulate robot IK
# Add a taskspace marker

In [61]:
roboDoD.plant.num_actuators()

3

In [ ]:
# Joint teleop
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()
   
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile('./test.urdf', 'lcl-DoD')
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame())
    plant.Finalize()

    
    endMod = roboDoD.plant.get_body(roboDoD.plant.GetBodyIndices(roboDoD.model)[-1]).name()
    # Draw the frames
    for body_name in ["root_mod", endMod]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

#     plant.SetPositions(plant.GetMyContextFromRoot(context),
#                   plant.GetModelInstanceByName("diva_robot"),
#                   [-3.54, -2.33, -1.49, -1.62, -0.97, -0.19, 0.47])
    
    gripper = plant.GetBodyByName(endMod)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()

gripper position (m): [0.00 -0.01 0.06]
gripper roll-pitch-yaw (rad):[0.80 -0.00 1.67]
pose rotation:  RotationMatrix([
  [-0.09904103659872801, -0.6932812461724049, 0.7138291019390333],
  [0.9950833498101802, -0.06900255469903196, 0.0710476908531001],
  [0.0, 0.7173560908995228, 0.6967067093471654],
])
[ 0.00128894 -0.00992112  0.0625    ]


In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()